In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from collections import defaultdict

# Constants
MAX_SENT_LENGTH = 30
MAX_SENTS = 50
EMBEDDING_DIM = 300
NUM_HEADS = 20
HEAD_SIZE = 20
DROPOUT_RATE = 0.2
NPRATIO = 4  # Number of negative samples per positive
BATCH_SIZE = 64
EPOCHS = 5

In [5]:
# Load Datasets
articles = pd.read_csv("Data/articles.csv")
behaviors_train = pd.read_csv("Data/behaviors_train.csv")
behaviors_val = pd.read_csv("Data/behaviors_val.csv")
history_train = pd.read_csv("Data/history_train.csv")
history_val = pd.read_csv("Data/history_val.csv")

In [6]:
def build_vocab_and_tokenize(titles, max_len=MAX_SENT_LENGTH):
    """
    Builds a vocabulary and tokenizes article titles.
    
    Args:
        titles (list of str): List of article titles to tokenize.
        max_len (int): Maximum length for tokenized titles (truncation/padding length).
    
    Returns:
        tokenized_titles (list of list of int): Tokenized and padded titles.
        vocab (dict): A dictionary mapping tokens to unique integer indices.
        vocab_size (int): Size of the vocabulary.
    """
    vocab = defaultdict(lambda: len(vocab))  # Default dictionary for token ids
    vocab["<PAD>"] = 0  # Padding token
    vocab["<UNK>"] = 1  # Unknown token

    tokenized_titles = []
    for title in titles:
        tokens = title.lower().split()[:max_len]  # Simple whitespace tokenizer
        tokenized = [vocab[token] for token in tokens]
        padded = pad_sequence_to_length(tokenized, max_len, pad_value=vocab["<PAD>"])
        tokenized_titles.append(padded)

    # Freeze the vocabulary after processing to get accurate vocab size
    vocab = dict(vocab)  # Convert to a regular dict to freeze it
    vocab_size = len(vocab)

    return tokenized_titles, vocab, vocab_size

def pad_sequence_to_length(sequence, target_length, pad_value=0):
    """
    Pads or truncates a sequence to the specified target length.
    
    Args:
        sequence (list of int): Input sequence to pad or truncate.
        target_length (int): Desired length of the sequence.
        pad_value (int): Value to use for padding shorter sequences.
    
    Returns:
        list of int: Padded or truncated sequence.
    """
    if len(sequence) >= target_length:
        return sequence[:target_length]
    else:
        return sequence + [pad_value] * (target_length - len(sequence))

In [7]:
# Tokenize titles and build vocabulary
articles["tokenized_title"], vocab, VOCAB_SIZE = build_vocab_and_tokenize(
    articles["title"].fillna("<UNK>"),
    max_len=MAX_SENT_LENGTH
)

In [8]:
history_train

,Unnamed: 0,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,0,13538,['2023-04-27T10:17:43.000000' '2023-04-27T10:1...,[100. 35. 100. 24. 100. 23. 100. 100. 100. ...,[9738663 9738569 9738663 9738490 9738663 97386...,[ 17. 12. 4. 5. 4. 9. 5. 46. 11. ...
1,1,58608,['2023-04-27T18:48:09.000000' '2023-04-27T18:4...,[ 37. 61. 100. 100. 55. 100. 100. nan 61. ...,[9739362 9739179 9738567 9739344 9739202 97393...,[ 2. 24. 72. 65. 11. 4. 101. 0. 699. ...
2,2,95507,['2023-04-27T15:20:28.000000' '2023-04-27T15:2...,[ 60. 100. 100. 21. 29. 67. 49. 54. 25. ...,[9739035 9738646 9634967 9738902 9735495 97373...,[ 18. 29. 51. 12. 10. 10. 13. 2...
3,3,106588,['2023-04-27T08:29:09.000000' '2023-04-27T08:2...,[ 24. 57. 100. nan nan 100. 100. 73. 26. ...,[9738292 9738216 9737266 9737556 9737657 97377...,[9.000e+00 1.500e+01 4.200e+01 9.000e+00 3.000...
4,4,617963,['2023-04-27T14:42:25.000000' '2023-04-27T14:4...,[100. 100. nan 46. 23. 19. 61. 70. 64. ...,[9739035 9739088 9738902 9738968 9738760 97389...,[ 45. 29. 116. 26. 34. 42. 58. 5...
...,...,...,...,...,...,...
1585,1585,1122370,['2023-05-09T06:50:19.000000' '2023-05-09T06:5...,[100. 100. 100. 100. nan],[9755181 9706958 9709329 9706958 9755571],[ 84. 115. 298. 5. 79.]
1586,1586,1718049,['2023-05-17T05:04:36.000000' '2023-05-17T05:0...,[ 53. 100. 24. 100. 83. 28. 37. 40. 22.],[9768583 9728166 9768829 9769641 9769994 97696...,[13. 17. 7. 99. 5. 8. 4. 2. 4.]
1587,1587,1178033,['2023-04-29T19:36:47.000000' '2023-04-29T19:3...,[nan 98. 92. 30. 35. 9. 34. 58. nan],[9742268 9742423 9742270 9742440 9742459 97388...,[ 10. 74. 89. 10. 1. 4. 11. 7. 159.]
1588,1588,395912,['2023-05-12T05:12:34.000000' '2023-05-12T05:1...,[100. 100. 100. 33. nan],[9760741 9760998 9760935 9754442 9760935],[16. 22. 23. 9. 0.]


In [9]:
import logging

# Setup logging (if not already configured in your project)
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

def clean_article_ids(article_ids):
    """
    Cleans and parses article IDs from a string representation to a list of integers.
    
    Args:
        article_ids (str): String representation of article IDs (e.g., "[1, 2, 3]").
    
    Returns:
        list of int or None: List of parsed article IDs, or None if input is invalid.
    """
    try:
        # Check for invalid placeholders or empty strings
        if not article_ids or "..." in article_ids:
            logging.warning(f"Malformed article_ids found: '{article_ids}'. Skipping.")
            return None
        
        # Remove brackets and split on spaces or commas, then convert to integers
        cleaned_ids = article_ids.strip("[]").replace(",", " ").split()
        return list(map(int, cleaned_ids))
    except Exception as e:
        logging.error(f"Failed to clean article_ids '{article_ids}' due to error: {e}")
        return None

# Function to process a single dataset (train or val)
def clean_and_report_history(history_df, dataset_name="dataset"):
    """
    Cleans article IDs in the user history dataset and reports cleaning statistics.

    Args:
        history_df (pd.DataFrame): Input dataset with article history to clean.
        dataset_name (str): Name of the dataset (for logging and reporting).

    Returns:
        pd.DataFrame: Cleaned dataset with invalid rows removed.
    """
    # Clean article IDs
    history_df["cleaned_article_ids"] = history_df["article_id_fixed"].apply(clean_article_ids)

    # Count skipped rows
    skipped_rows = history_df["cleaned_article_ids"].isna().sum()
    total_rows = len(history_df)
    print(f"{dataset_name}: Skipped {skipped_rows} rows out of {total_rows} "
          f"({skipped_rows / total_rows:.2%}).")

    # Save problematic rows
    invalid_rows = history_df[history_df["cleaned_article_ids"].isna()]
    invalid_rows_file = f"invalid_article_ids_{dataset_name}.csv"
    invalid_rows.to_csv(invalid_rows_file, index=False)
    print(f"{dataset_name}: Saved {len(invalid_rows)} problematic rows to '{invalid_rows_file}'.")

    # Drop invalid rows and reset index
    cleaned_df = history_df.dropna(subset=["cleaned_article_ids"]).reset_index(drop=True)
    print(f"{dataset_name}: Remaining rows after cleaning: {len(cleaned_df)}")

    return cleaned_df

# Clean train and validation datasets
history_train_cleaned = clean_and_report_history(history_train, dataset_name="train")
history_val_cleaned = clean_and_report_history(history_val, dataset_name="val")


def process_cleaned_user_history(cleaned_history_df):
    user_histories = defaultdict(list)
    for _, row in cleaned_history_df.iterrows():
        user_id = row["user_id"]
        article_ids = row["cleaned_article_ids"]
        user_histories[user_id].extend(article_ids)
    return user_histories

user_history_train_cleaned = process_cleaned_user_history(history_train_cleaned)
user_history_val_cleaned = process_cleaned_user_history(history_val_cleaned)

2024-12-07 15:51:57,536 - WARNING - Malformed article_ids found: '[9738366 9737535 9738173 ... 9766140 9766140 9766140]'. Skipping.
2024-12-07 15:51:57,541 - WARNING - Malformed article_ids found: '[9737083 9737083 9738216 ... 9770037 9769994 9768321]'. Skipping.


train: Skipped 2 rows out of 1590 (0.13%).
train: Saved 2 problematic rows to 'invalid_article_ids_train.csv'.
train: Remaining rows after cleaning: 1588
val: Skipped 0 rows out of 1562 (0.00%).
val: Saved 0 problematic rows to 'invalid_article_ids_val.csv'.
val: Remaining rows after cleaning: 1562


In [10]:
print(f"Vocabulary Size: {VOCAB_SIZE}")
print(f"Sample Tokens: {list(vocab.keys())[:10]}")

Vocabulary Size: 18591
Sample Tokens: ['<PAD>', '<UNK>', 'ishockey-spiller:', 'jeg', 'troede', 'skulle', 'dø', 'prins', 'harry', 'tvunget']


In [11]:
import json
import os

data_folder = "Data"
vocab_file = os.path.join(data_folder, "vocab.json")

os.makedirs(data_folder, exist_ok=True)

# Save the vocabulary to a file
with open(vocab_file, "w") as f:
    json.dump(vocab, f)

print(f"Vocabulary saved to {vocab_file}")

Vocabulary saved to Data/vocab.json


In [12]:
from random import sample

def create_samples(behaviors_df, user_history, npratio=NPRATIO, max_sents=MAX_SENTS):
    samples = []
    labels = []
    for _, row in behaviors_df.iterrows():
        user_id = row["user_id"]

        clicked_articles = clean_article_ids(row['article_ids_clicked'])
        inview_articles = clean_article_ids(row['article_ids_inview'])

        if clicked_articles is None or inview_articles is None:
            continue

        # Fetch, truncate, and pad user history
        user_hist = user_history.get(user_id, [])
        if len(user_hist) > max_sents:
            user_hist = user_hist[:max_sents]
        if len(user_hist) < max_sents:
            user_hist += [0] * (max_sents - len(user_hist))

        # Positive sampling
        for article_id in clicked_articles:
            article_id = min(article_id, VOCAB_SIZE - 1) # Ensire valid index
            samples.append((user_hist, article_id))
            labels.append(1)

        # Negative sampling
        negative_articles = list(set(inview_articles) - set(clicked_articles))
        if len(negative_articles) > npratio * len(clicked_articles):
            negative_articles = sample(negative_articles, npratio * len(clicked_articles))
        for article_id in negative_articles:
            article_id = min(article_id, VOCAB_SIZE - 1)  # Ensure valid index
            samples.append((user_hist, article_id))
            labels.append(0)

    return samples, labels

train_samples_cleaned, train_labels_cleaned = create_samples(
    behaviors_train, user_history_train_cleaned, npratio=NPRATIO
    )
val_samples_cleaned, val_labels_cleaned = create_samples(
    behaviors_val, user_history_val_cleaned, npratio=NPRATIO
    )

In [ ]:

# Step 4: Define PyTorch Dataset
class NewsDataset(Dataset):
    def __init__(self, samples, labels):
        self.samples = samples
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        user_history, candidate = self.samples[idx]
        user_history_padded = [0] * MAX_SENTS  # Pad user history
        user_history_padded[:len(user_history)] = user_history[:MAX_SENTS]
        candidate_padded = [candidate] + [0] * (MAX_SENT_LENGTH - 1)  # Pad candidate to sequence
        return (
            torch.tensor(user_history_padded, dtype=torch.long),
            torch.tensor(candidate_padded, dtype=torch.long),
            torch.tensor(self.labels[idx], dtype=torch.long),
        )

train_dataset = NewsDataset(train_samples_cleaned, train_labels_cleaned)
val_dataset = NewsDataset(val_samples_cleaned, val_labels_cleaned)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
# Step 5: Define the NRMS Model
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = head_size
        self.output_dim = num_heads * head_size
        self.qkv_linear = nn.Linear(EMBEDDING_DIM, self.output_dim * 3)
        self.fc_out = nn.Linear(self.output_dim, EMBEDDING_DIM)

    def forward(self, x):
        if len(x.size()) != 3:
            raise ValueError(f"Expected input to be 3D (batch_size, seq_length, embed_dim), got {x.size()}")
        batch_size, seq_length, embed_dim = x.size()
        qkv = self.qkv_linear(x).reshape(batch_size, seq_length, self.num_heads, 3 * self.head_size)
        qkv = qkv.permute(2, 0, 1, 3)
        Q, K, V = torch.chunk(qkv, 3, dim=-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / np.sqrt(self.head_size)
        attention = F.softmax(scores, dim=-1)
        weighted = torch.matmul(attention, V)
        return self.fc_out(weighted.permute(1, 2, 0, 3).reshape(batch_size, seq_length, self.output_dim))

class TitleEncoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(DROPOUT_RATE)
        self.self_attention = MultiHeadSelfAttention(NUM_HEADS, HEAD_SIZE)
        self.dense = nn.Linear(embedding_dim, 1)

    def forward(self, x):
        x = self.embedding(x)  # Ensure input is embedded
        if len(x.size()) != 3:
            raise ValueError(f"Embedding layer output should be 3D, got {x.size()}")
        x = self.dropout(x)
        x = self.self_attention(x)
        attention_weights = F.softmax(self.dense(x).squeeze(-1), dim=-1)
        return torch.sum(x * attention_weights.unsqueeze(-1), dim=1)

class NRMS(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        super().__init__()
        self.title_encoder = TitleEncoder(vocab_size, embedding_dim)

    def forward(self, candidates, user_history):
        user_rep = self.title_encoder(user_history)  # Output: (batch_size, embedding_dim)
        candidate_rep = self.title_encoder(candidates)  # Output: (batch_size, embedding_dim)
        return torch.matmul(candidate_rep, user_rep.unsqueeze(-1)).squeeze(-1)

In [10]:
# Step 6: Train the Model
model = NRMS(VOCAB_SIZE, EMBEDDING_DIM, NPRATIO + 1)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

def train_model(model, train_loader, val_loader, optimizer, criterion, epochs=EPOCHS):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            user_histories, candidates, labels = batch
            optimizer.zero_grad()
            outputs = model(candidates, user_histories)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

In [ ]:
# Train the model
train_model(model, train_loader, val_loader, optimizer, criterion)

Epoch 1, Loss: 20812.2647
